In [ ]:
!pip install -q -U transformers onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 669.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained('norsu/ai-detector-test')
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [ ]:
import torch
import time
model.eval()
start = time.time()
input = tokenizer("asdsf", return_tensors='pt')
with torch.inference_mode():
  print(torch.softmax(model(**input)[0],1))

total = time.time() - start
total

tensor([[8.8574e-04, 9.9911e-01]])


3.8182132244110107

In [ ]:
from transformers import pipeline

pipe = pipeline('text-classification', model, tokenizer=tokenizer)

In [ ]:
start = time.time()
pipe("asdsf")
time.time() - start

0.5589210987091064

In [ ]:
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [23]:
import onnx

batch_size = 32


with torch.inference_mode():
    inputs = {"input_ids":torch.ones(1, 768, dtype=torch.int64),
     "attention_mask":torch.ones(1, 768, dtype=torch.int64),}
    outputs = model(**inputs)
    symbolic_names = {0:'batch_size', 1:'max_seq_len'}
    torch.onnx.export(model,
                (inputs['input_ids'],
                  inputs['attention_mask'],),
                      r"lean-ai-detector.onnx",
                      opset_version=12,
                      input_names=['input_ids',
                                   'input_mask',],
                      output_names=['output'],
                      dynamic_axes={'input_ids': symbolic_names,
                                    'input_mask': symbolic_names,})



In [24]:
onnx_model_path = "lean-ai-detector.onnx"
quantized_model_path = "Q-lean-ai-detector.onnx"
from onnxruntime.quantization import quantize_dynamic, QuantType
onnx_opt_model = onnx.load(onnx_model_path)
quantize_dynamic(onnx_model_path,
                quantized_model_path,
                weight_type=QuantType.QInt8)

In [25]:
import os

print('ONNX full precision model size (MB):', os.path.getsize("lean-ai-detector.onnx")/(1024*1024))
print('ONNX quantized model size (MB):', os.path.getsize("Q-lean-ai-detector.onnx")/(1024*1024))

ONNX full precision model size (MB): 704.3488445281982
ONNX quantized model size (MB): 232.57878398895264


In [26]:
import onnxruntime

model_path = "Q-lean-ai-detector.onnx"

sess_options = onnxruntime.SessionOptions()
session = onnxruntime.InferenceSession(model_path, sess_options)

In [41]:
a = tokenizer("asdf", return_tensors="np")

ort_inputs = {
                    'input_ids':  a['input_ids'],
                    'input_mask': a['attention_mask'],
                }

In [42]:
import numpy as np

logits = np.reshape(session.run(None, ort_inputs), (-1,2))

In [46]:
np.argmax(logits)

1